In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import sys
sys.path.append("..")
from simdb.databaseModel import *
from simdb.databaseAPI import *

In [ ]:
# db_path = 'test_WORKS.db'
db_path = '../examples/setup_databases/andrej_raw.db'

## For migration to databaseAPI:

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [ ]:
class DatabaseConnection(object):
    
    def __init__(self, db_path):
        self.db_path = db_path
        self.session = None
        

    def __enter__(self):
        # make a database connection and return it
        if not os.path.exists(self.db_path):
            raise "%s does not exist."
        
        engine = create_engine('sqlite:///{}'.format(self.db_path))
        Session = sessionmaker()
        self.session = Session(bind=engine)
        
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.session.close()

In [ ]:

class SQLAlchemyDBConnection(object):
    """SQLAlchemy database connection"""

    def __init__(self, connection_string):
        self.connection_string = connection_string
        self.session = None

    def __enter__(self):
        engine = create_engine(self.connection_string)
        Session = sessionmaker()
        self.session = Session(bind=engine)
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.session.close()

In [ ]:
with DatabaseConnection(db_path) as db:
    print db.session.query(Keywords)

In [ ]:
q = s.session.query(Keywords)
[e.name for e in q.filter(Keywords.value != None).all()]

In [ ]:
s.close()

In [ ]:
s.add(Main())

In [ ]:
s.commit()

In [ ]:
s.close()

## HowTo

### Load Entry Table

In [ ]:
table = getEntryTable(db_path, load_keys=True, load_tags=True) # load whole table

In [ ]:
table

### Obtain data for certain entry

In [ ]:
entry_id = "2017_08_01_2ub_k27_02_06"

In [ ]:
getEntryDetails(db_path, entry_id)

In [ ]:
getEntryMeta(db_path, entry_id)

In [ ]:
getEntryKeywords(db_path, entry_id)

In [ ]:
getEntryTags(db_path, entry_id)

### Select entries

In [ ]:
table[selectByTag(table, "test")] # select by tag

In [ ]:
table[selectByKeyword(table, "ff", "MARTINI")] # select by keyword

In [ ]:
# combinations of selections
ff       = selectByKeyword(table, "ff", "MARTINI")
chainlen = selectByKeyword(table, "chainlen", "1")
tags     = selectByTag(table, "test2")

table[ff & chainlen & tags]

In [ ]:
# another example
ff       = selectByKeyword(table, "ff", "MARTINI")
chainlen = selectByKeyword(table, "chainlen", "2")
linker1  = selectByKeyword(table, "linker", "k48")
linker2  = selectByKeyword(table, "linker", "k63")
number   = selectByKeyword(table, "number", "01")

table[ff & chainlen & (linker1 | linker2) & ~number]

### Usefull for backend / viewer

In [ ]:
# open database (in SQLAlchemy we open a session)
# s = openDatabase(db_path) 
# one should not keep the DB open

# better:
with openDatabase(db_path) as db:
    print db
    

In [ ]:
getTags(db_path)  # get all tags which are used in DB

In [ ]:
keywords = getKeywords(db_path)  # get all keyword with the values they have
pd.Series(keywords) 

### Running functions

## Deva

* Add tags/keywords to all simulations in pandas table
* Add simulations to DB from pandas table